In [ ]:
import boto3
from botocore.client import Config

# Initialize S3 resource for MinIO
minio_s3 = boto3.resource(
    's3',
    endpoint_url='https://pangeo-eosc-minioapi.vm.fedcloud.eu',  # MinIO API endpoint:contentReference[oaicite:4]{index=4}
    aws_access_key_id=MINIO_ACCESS_KEY,
    aws_secret_access_key=MINIO_SECRET_KEY,
    region_name='us-east-1',  # or any region; required by boto3 but not used by MinIO
    config=Config(signature_version='s3v4')  # Use V4 signatures for MinIO:contentReference[oaicite:5]{index=5}
)
source_bucket = s3.Bucket("eodata")               # source bucket (Copernicus)
dest_bucket   = minio_s3.Bucket("okihle-s2l1b")   # destination bucket (MinIO)

In [ ]:
prefix = "Sentinel-2/MSI/MSI_L1B_GR/2025/07/24/"
for obj in source_bucket.objects.filter(Prefix=prefix):
    if obj.key.endswith("/"):
        continue  # skip folder placeholders
    print(f"Copying {obj.key} ...")
    # Stream from source and upload to destination
    source_response = source_bucket.Object(obj.key).get()
    dest_bucket.upload_fileobj(source_response['Body'], obj.key)